In [4]:
import numpy as np
import pandas as pd

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [6]:
df = pd.read_csv('covid_toy.csv')

In [7]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [8]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [9]:
df.isnull().sum() # fever column has missing values

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop(columns =['has_covid']), df['has_covid'], test_size = 0.2)

In [12]:
X_train

,age,gender,fever,cough,city
30,15,Male,101.0,Mild,Delhi
81,65,Male,99.0,Mild,Delhi
32,34,Female,101.0,Strong,Delhi
27,33,Female,102.0,Strong,Delhi
77,8,Female,101.0,Mild,Kolkata
...,...,...,...,...,...
85,16,Female,103.0,Mild,Bangalore
57,49,Female,99.0,Strong,Bangalore
66,51,Male,104.0,Mild,Kolkata
15,70,Male,103.0,Strong,Kolkata


# preprocessing without using Column Transformer

In [13]:
# adding SimpleImputer to fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
# also fit_transform the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape
# all the missing values are being replaced by the mean of the data

(80, 1)

In [14]:
# OrdinalEncoder -> cough
oe = OrdinalEncoder(categories = [['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape


(80, 1)

In [15]:
# OneHotEncoder -> gender, city
ohe = OneHotEncoder(drop = 'first', sparse_output = False)
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])

X_test_gender_city = ohe.fit_transform(X_test[['gender', 'city']])
X_train_gender_city

array([[1., 1., 0., 0.],
       [1., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 0., 0., 1.],
       [1., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 1.],
       [0., 0., 0., 0.],
       [1., 0., 0., 1.],
       [1., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 0., 0.],


In [16]:
# Ectracting age
X_train_age = X_train.drop(columns = ['gender','fever', 'cough', 'city']).values

#also the test data
X_test_age = X_test.drop(columns = ['gender', 'fever', 'cough', 'city' ]).values

X_train_age.shape


(80, 1)

In [17]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis = 1)

# same for the test data
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis = 1)
X_train_transformed.shape

(80, 7)

# Now same data transformation using column transformer

In [18]:
from sklearn.compose import ColumnTransformer

In [19]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories = [['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output = False, drop = 'first'), ['gender', 'city'])
], remainder = 'passthrough')


In [20]:
transformed_train_transformer = transformer.fit_transform(X_train)

In [21]:
transformed_train_transformer

array([[101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  15.        ],
       [ 99.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  65.        ],
       [101.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  34.        ],
       [102.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  33.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   8.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  27.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  73.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  24.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.    

In [22]:
transformed_train_transformer.shape

(80, 7)

In [23]:
transformed_test_transformer = transformer.fit_transform(X_test)

In [24]:
transformed_test_transformer

array([[100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  10.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  55.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  34.        ],
       [103.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  46.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [100.33333333,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  42.        ],
       [102.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  49.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  24.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.    

In [25]:
transformed_test_transformer.shape

(20, 7)